In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
! cp /content/drive/My\ Drive/DatasetForHackabit2019/Hollywood-dev/movies/imgs.zip /content/
! unzip /content/imgs.zip

In [0]:
t = [[26871, 30386],
     [77509, 79892],
     [81790, 87836],
     [131366, 132776],
     [135979, 157150],
     [161739, 163247],
     [166900, 168463],
     [177235, 178336],
     [184014, 185293],
     [188466, 188504],
     [198250, 213895]] # 9261
# t = 9249 14986
# 16441 25730
# 31113 36998
# 69574 70219
# 75703 80256
# 84463 85400
# 126263 152423
# 191603 210122
# 212196 223246

path = '/content/imgs'
os.mkdir(os.path.join(path, '0'))
os.mkdir(os.path.join(path, '1'))

import os
for i in range(1, 9261):
    chngd = False
    for j in t:
        if i>=round(j[0]/23.98) and i<=round(j[1]/23.98):
            tp = '1'
            chngd = True
            break
    if not chngd:
        tp = '0'
    os.rename(os.path.join(path, 'p'+str(i)+'.png'),
              os.path.join(path, tp, 'p'+str(i)+'.png'))

In [0]:
import keras.layers as klr
from keras.models import Model
import keras.applications as kapp
import keras.preprocessing.image as kimg
from keras import callbacks
import numpy as np
import matplotlib.pyplot as plt

In [0]:
model = kapp.Xception(input_shape=(240, 240, 3), include_top=False, weights='imagenet')

x = model.output
x = klr.GlobalAveragePooling2D()(x)
x = klr.Dense(2, activation='softmax')(x)
model = Model(model.input, x)
model.summary()

In [17]:
for layer in model.layers:
    layer.trainable = True

model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 240, 240, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 119, 119, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 119, 119, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 119, 119, 32) 0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


In [0]:
train_data = kimg.ImageDataGenerator(rescale=1/255).flow_from_directory('/content/imgs', target_size=(240, 240))

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [0]:
checkpointer = callbacks.ModelCheckpoint(filepath='/content/drive/My Drive/weight/weights.hdf5', verbose=1, save_best_only=True)
hist = model.fit_generator(train_data, epochs=5, callbacks=[checkpointer])
plt.plot(hist.history['loss'])

In [0]:
model.save_weights('/content/drive/My Drive/weight/weights.hdf5')